In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
import seaborn as sns

In [2]:
# # Read it in.
# # post = pd.read_excel('../data/postscndryunivsrvy2013dirinfo_cut.xlsx')
# peps = pd.read_excel('../data/peps300.xlsx')

# # Do some fixing and plumbing
# post['OPEID_CLEAN'] = [value[2:] for value in post['OPEID_CLEAN'].values]
# peps['OPEID_CLEAN'] = [str(value) for value in peps['OPEID_CLEAN'].values]

# # Do some more plumbing.
# peps = peps.iloc[:, :20]

# # Merge our datasets.
# joined = pd.merge(post, peps, on='OPEID_CLEAN')

FileNotFoundError: [Errno 2] No such file or directory: '../data/postscndryunivsrvy2013dirinfo_cut.xlsx'

In [4]:
# Read it in.
survey2011 = pd.read_csv('../data/hd2011.csv', encoding = "ISO-8859-1")
survey2012 = pd.read_csv('../data/hd2012.csv', encoding = "ISO-8859-1")
peps = pd.read_excel('../data/peps300.xlsx')

In [5]:
# Do some fixing and plumbing
#survey2011['OPEID_CLEAN'] = [value[2:] for value in post['OPEID'].values]  // I don't know why you guys are doing this, so I just choose not to use it for now
#peps['OPEID_CLEAN'] = [str(value) for value in peps['OPEID_CLEAN'].values]

# pepsG = the first 11 and columns which we used for both train and test
pepsG = peps.iloc[:, :11]

pepsG['OPEID'] = pepsG['OPEID_CLEAN']

pepsG = pepsG.drop('OPEID_CLEAN', axis =1)

pepsG['Ethnic Code'] = peps['Ethnic Code']

pepsG['Average or\nGreater than 30'] = peps['Average or\nGreater than 30']


#prepare the train set/2011
peps11 = peps.iloc[:, 27:31]
train = pd.merge(pepsG, peps11, left_index = True, right_index=True)

# #prepare the test set/2012
peps12 = peps.iloc[:, 22:26]
test = pd.merge(pepsG, peps12,left_index = True, right_index=True)

# # Merge our datasets.
train = pd.merge(survey2011, train, on='OPEID')
test = pd.merge(survey2012, test, on='OPEID')

In [11]:
train.columns.values

array(['UNITID', 'INSTNM', 'ADDR', 'CITY', 'STABBR', 'ZIP', 'FIPS',
       'OBEREG', 'CHFNM', 'CHFTITLE', 'GENTELE', 'EIN', 'OPEID', 'OPEFLAG',
       'WEBADDR', 'ADMINURL', 'FAIDURL', 'APPLURL', 'NPRICURL', 'SECTOR',
       'ICLEVEL', 'CONTROL', 'HLOFFER', 'UGOFFER', 'GROFFER', 'HDEGOFR1',
       'DEGGRANT', 'HBCU', 'HOSPITAL', 'MEDICAL', 'TRIBAL', 'LOCALE',
       'OPENPUBL', 'ACT', 'NEWID', 'DEATHYR', 'CLOSEDAT', 'CYACTIVE',
       'POSTSEC', 'PSEFLAG', 'PSET4FLG', 'RPTMTH', 'IALIAS', 'INSTCAT',
       'CCBASIC', 'CCIPUG', 'CCIPGRAD', 'CCUGPROF', 'CCENRPRF', 'CCSIZSET',
       'CARNEGIE', 'TENURSYS', 'LANDGRNT', 'INSTSIZE', 'CBSA', 'CBSATYPE',
       'CSA', 'NECTA', 'F1SYSTYP', 'F1SYSNAM', 'FAXTELE', 'COUNTYCD',
       'COUNTYNM', 'CNGDSTCD', 'LONGITUD', 'LATITUDE', 'Name', 'Address',
       'City', 'State', 'State Desc', 'Zip Code', 'Zip Ext',
       'Prog\nLength', 'School\nType', 'Ethnic Code',
       'Average or\nGreater than 30', 'Dual\nNum 3', 'Dual\nDenom 3',
       'DRate 3'

In [12]:
# ASK BRIAN:
#  CBSA: population density measure
#    CBSATYPE
#    CSA
#  Congressional district

# Not including geo-location features like zip code, county.
# TODO: revisit Latitude and Longitude for visualization.

features = ['UNITID',
            'CONTROL',
            'GROFFER',
            'HBCU',
            'HLOFFER',
            'HOSPITAL',
            'ICLEVEL',
            'INSTCAT',
            'INSTSIZE',
            'LANDGRNT', # Revisit
            'LOCALE',
            'PSEFLAG',
            'SECTOR',
            'Prog\nLength',
            'School\nType',
#             '\nDenom 1',
#             '\nNum 1',
            'UGOFFER',
#             'DRate 3',
            'Ethnic Code',
            'Average or\nGreater than 30']

med = np.median(train['DRate 3'])

# ax = plt.axes()
# ax.set_title("KDE for Default Rate")
# kde = sns.kdeplot(joined['DRate 1'], bw = 0.4)
# fig = kde.get_figure()
# fig.savefig("kde_plot.png")

In [18]:
# Master dataset!!
# joined = joined[features]

train_features = features + ['DRate 3']
test_features = features + ['DRate 2']

train = train[train_features]
test = test[test_features]

train['DRate'] = train['DRate 3']
test['DRate'] = test['DRate 2']
train = train.drop('DRate 3', axis = 1)
test = test.drop('DRate 2', axis = 1)

# np.median(joined['DRate 1']) == 9.80000
# med = np.median(train['DRate 3'])
med = np.median(train['DRate'])

# joined['DRate_Transformed'] = joined['DRate 1'] > med
# joined['DRate_Transformed'] = joined['DRate_Transformed'].astype(int)

train['DRate_Transformed'] = train['DRate'] > med
train['DRate_Transformed'] = train['DRate_Transformed'].astype(int)

# TODO eventually re-add this statement
# joined = joined.drop('DRate 1', axis = 1)

/Users/danny/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
# Our initial model works with THIS dataset.
# joined.to_csv("../data/peps_to_post.csv", index=False)
train.to_csv("../data/joined_train.csv", index=False)
test.to_csv("../data/joined_test.csv", index=False)

In [20]:
ipeds = pd.read_csv("../data/delta_public_00_12.csv", encoding="ISO-8859-1")

/Users/danny/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
# ipeds['is_2012'] = ipeds['academicyear'] == 2012
# ipeds = ipeds[ipeds['is_2012']]

ipeds['is_2011'] = ipeds['academicyear'] == 2011
ipeds2011 = ipeds[ipeds['is_2011']]

ipeds['is_2012'] = ipeds['academicyear'] == 2012
ipeds2012 = ipeds[ipeds['is_2012']]

In [22]:
# REMOVE instname, unitid WHEN DONE

ipeds_features ="""
instname
unitid
sector
iclevel
control
census_region
hbcu
hsi
cpi_index
hepi_index
has_fte
nettuition01
net_student_tuition
endowment03
priv_invest_endow
total03_revenue
restricted_revenue
grant01
grant02
grant07
tuition_discount
institutional_grant_aid_share
fed_grant_pct
inst_grant_pct
loan_pct
loan_avg_amount
inst_grant_avg_amount
tuitionfee02_tf
studserv01
acadsupp01
sticker_price_share
nettuition_share
totaldegrees_100fte
certificates_awards_100fte
totalcompletions_100fte
ft_first_time_first_yr_deg_seek
total_full_time
total_enrollment_asian_tot
total_enrollment_black_tot
total_enrollment_hisp_tot
total_enrollment_white_tot
ft_faculty_per_100fte
pt_faculty_per_100fte
salarytotal""".split()

# Isolate wanted columns, map column headers to uppercase for join.
# ipeds = ipeds[ipeds_features]
# ipeds.columns = map(str.upper, ipeds.columns)
ipeds2011 = ipeds2011[ipeds_features]
ipeds2012 = ipeds2012[ipeds_features]
ipeds2011.columns = map(str.upper, ipeds2011.columns)
ipeds2012.columns = map(str.upper, ipeds2012.columns)

In [23]:
# Merge :joined and :ipeds on the UNITID column.
# master = pd.merge(joined, ipeds, on="UNITID")

train = pd.merge(train, ipeds2011, on="UNITID")
test = pd.merge(test, ipeds2012, on="UNITID")

In [24]:
# master[master['UNITID'] == 103811].INSTNAME
train[train['UNITID'] == 103811].INSTNAME

49    American Institute of Technology
Name: INSTNAME, dtype: object

In [25]:
# Set our institution's UNITID to be its index, drop
# the corresponding UNITID column.
# master.index = master['UNITID']
# master = master.drop("UNITID", axis = 1)

train.index = train['UNITID']
train = train.drop("UNITID", axis = 1)

test.index = test['UNITID']
test = test.drop("UNITID", axis = 1)

# Leave this here please :)
# master_copy = master
# master_copy.index = master_copy['INSTNAME']
# master_copy = master_copy.drop("INSTNAME", axis = 1)

In [62]:
# Master dataset! Woo.
# master.to_csv("../data/master.csv")
train.to_csv("../data/master_train.csv")

# Design school with NaN DRate
test = test.drop([134680,456296])

test.to_csv("../data/master_test.csv")

In [ ]:
pd.isnull(master).any()
nans = pd.isnull(master)
nans_copy = pd.isnull(master_copy)

from collections import Counter

university_nans = Counter()
column_count_nans = Counter()

for index, row in zip(nans_copy.index, nans_copy.values):
    row_data = dict(zip(nans_copy.columns.values, row))

    for column, value in row_data.items():
        if value:
            university_nans[index] += 1
            column_count_nans[column] += 1

In [ ]:
# A good number of our institutions have between 0 and 10 NaN values.
university_nan_dist = pd.DataFrame.from_dict(university_nans, orient='index').reset_index()
university_nan_dist = university_nan_dist.rename(columns={'index':'event', 0:'count'})
plt.hist(university_nan_dist['count'])
plt.show()

In [ ]:
column_nan_dist = pd.DataFrame.from_dict(column_count_nans, orient='index').reset_index()
column_nan_dist = column_nan_dist.rename(columns={'index':'event', 0:'count'})
column_nan_dist['count'] = column_nan_dist['count'].astype(int)

In [ ]:
# Plot new features against their NaN count

fig, ax = plt.subplots()
x = range(len(column_nan_dist['event']))
plt.xticks(x, column_nan_dist['event'])
ax.set_xticklabels(column_nan_dist['event'], rotation = 90)
plt.plot(x, column_nan_dist['count'], "g")
plt.show()

In [ ]:
# master = master.drop('ENDOWMENT03', axis = 1)

In [ ]:
# Isolate rows where master contains no missing values
not_nans = master.dropna(axis=0)

In [ ]:
# Get percent of valid rows in master
valid_percent_of_master = 1 - (master.shape[0] - not_nans.shape[0]) / master.shape[0]
valid_percent_of_master

In [ ]:
# Plot new default rate based on just the valid rows after dropping ENDOWMENT03
ax.set_title("KDE for Default Rate")
kde = sns.kdeplot(not_nans['DRate 1'], bw = 0.4)

In [ ]:
# Get the distribution median for this non-null dataset.
# Note that the median declined from 9.8 to 9.2
np.median(not_nans['DRate 1'])